In [21]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) # elimina los errres de que van a cambiar

# importamos las librerías que necesitamos
# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

# Imputación de nulos usando métodos avanzados estadísticos
# -----------------------------------------------------------------------
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

# Librerías de visualización
# -----------------------------------------------------------------------
import seaborn as sns
import matplotlib.pyplot as plt
# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

from word2number import w2n

In [22]:
!pip install word2number 
# from word2number import w2n 

In [30]:
class LimpiarDatosBBDD:
    def __init__(self):
        self.df = pd.read_csv("data/HR RAW DATA.csv")
    
    def cambiar_nombres_columnas (self):
        nombres_columnas = {"employeecount":"EmployeeCount",
                     "employeenumber" :"EmployeeNumber",
                     "NUMCOMPANIESWORKED" : "NumCompaniesWorked",
                     "TOTALWORKINGYEARS" :"TotalWorkingYears",
                     "WORKLIFEBALANCE" : "WorkLifeBalance",
                     "YEARSWITHCURRMANAGER" : "YearsWithCurrManager",
                     "NUMBERCHILDREN" : "NumberChildren", "Unnamed: 0": "ID"}
        self.df.rename(columns=nombres_columnas, inplace=True)
    
    def eliminar_columnas (self):
        columnas_eliminar = ["EmployeeCount", "Salary", "NumberChildren", "SameAsMonthlyIncome", "Over18", "RoleDepartament"]
        self.df.drop(columns=columnas_eliminar, inplace=True)

    def cambiar_a_numerico (self):
        # convertir la edad str en número:
        def convertir_edad(value):
            try:
                return w2n.word_to_num(value)
            except ValueError:
                return value  # Devolver el valor original si no puede ser convertido
        self.df["Age"] = self.df["Age"].apply(convertir_edad)
        
        # quitar simbolo $
        self.df["DailyRate"] = self.df["DailyRate"].str.replace("$", "")

        # Convertir la distancia al trabajo en positivo
        self.df["DistanceFromHome"] = self.df["DistanceFromHome"].abs()
            
        # columnas a convertir a numerico
        columnas_a_convertir = ["DailyRate", "EmployeeNumber", "MonthlyIncome", "TotalWorkingYears", "Age", "HourlyRate", "StandardHours"]
        for col in columnas_a_convertir:
            if self.df[col].dtype == 'object':
                # Verificar si la columna contiene comas antes de intentar reemplazarlas
                if self.df[col].str.contains(",").any():
                    self.df[col] = self.df[col].str.replace(",", ".")
            # Convertir la columna a formato numérico
                try:
                    self.df[col] = pd.to_numeric(self.df[col], errors='coerce')
                except:
                    self.df[col] = np.nan           
        
            
    def cambiar_a_categorica (self):
        # if self.df.select_dtypes(include = "object")
        # Comprobar las columnas que tiene guion y quitarlo
        columnas_guion = [col for col in self.df.select_dtypes(include = "object").columns if self.df[col].str.contains('-|_', regex=True).any()]        
        for col in columnas_guion:
            self.df[col] = self.df[col].apply(lambda val: val.replace("_", " ").replace("-", " ") if pd.notna(val) else val)

        # convertimos los nombres a minusculas y luego ponemos la primera letra en mayusculas
        columnas_objetivo = self.df.select_dtypes(include="object").columns
        for col in columnas_objetivo:
            self.df[col] = self.df[col].str.lower().str.title()

        # Quitamos el segundo digito del valor. Deberian de ser valores del 1 al 4 y este patron se repite solo en el primer digito, eliminamos el segundo.
        self.df['EnvironmentSatisfaction'] = self.df['EnvironmentSatisfaction'].apply(lambda num: num if num < 10 else num // 10 )

        # Poner nombres descriptivos               
        self.df["Education"].replace(to_replace= [1, 2, 3, 4, 5], value=["Primary", "Secondary", "High School", "Bachelor's", "Postgraduate"], inplace=True)
        self.df["JobLevel"].replace(to_replace= [1, 2, 3, 4, 5], value=["Entry Level", "Assistant", "Coordinator", "Manager", "Director"], inplace=True)
        self.df["Gender"].replace(to_replace= [0, 1], value=["Male", "Female"], inplace=True)
        self.df["RemoteWork"].replace(to_replace= ["0", "1", "True", "False"], value=["No", "Yes", "Yes", "No"], inplace=True)
        self.df["StockOptionLevel"].replace(to_replace= [0, 1, 2, 3], value=["None", "Low", "Medium", "High"], inplace=True)
        self.df["MaritalStatus"].replace(to_replace= "Marreid", value= "Married", inplace=True)
        
        satisfaccion = ["JobInvolvement", "EnvironmentSatisfaction", "JobSatisfaction", "RelationshipSatisfaction", "WorkLifeBalance"]
        for col in satisfaccion:
            self.df[col].replace(to_replace= [1, 2, 3, 4], value=["Very Low", "Low", "Medium", "High"], inplace=True)
    

    def guardar_archivo (self):
        # Save into a .csv file
        self.df.to_csv(f"./data/hr_raw_data_limpio_BBDD.csv")
       
           

In [31]:
limpieza = LimpiarDatos()
limpieza.cambiar_nombres_columnas()
limpieza.eliminar_columnas()
limpieza.cambiar_a_numerico()
limpieza.cambiar_a_categorica()
limpieza.guardar_archivo()

In [32]:
limpieza.df.head()

,ID,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeNumber,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NumCompaniesWorked,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,DateBirth,RemoteWork
0,0,51,No,NaN,684.0,NaN,6,High School,NaN,162.0,Very Low,Male,51.0,Medium,Director,Research Director,Medium,NaN,19537.0,6462,7,No,13,"3,0",Medium,NaN,None,NaN,5,"3,0",20,NaN,15,15,1972,Yes
1,1,52,No,NaN,699.0,NaN,1,Bachelor's,Life Sciences,259.0,Medium,Male,65.0,Low,Director,Manager,Medium,NaN,19999.0,5678,0,NaN,14,"3,0",Very Low,NaN,Low,34.0,5,"3,0",33,NaN,11,9,1971,Yes
2,2,42,No,Travel Rarely,532.0,Research & Development,4,Secondary,Technical Degree,319.0,Medium,Male,58.0,Medium,Director,Manager,High,Married,19232.0,4933,1,No,11,"3,0",High,NaN,None,22.0,3,NaN,22,NaN,11,15,1981,Yes
3,3,47,No,Travel Rarely,359.0,NaN,2,Bachelor's,Medical,NaN,Very Low,Female,82.0,Medium,Manager,Research Director,Medium,Married,17169.0,26703,3,NaN,19,"3,0",Low,NaN,Medium,NaN,2,NaN,20,NaN,5,6,1976,No
4,4,46,No,NaN,1319.0,NaN,3,High School,Technical Degree,NaN,Very Low,Female,45.0,High,Manager,Sales Executive,Very Low,Divorced,NaN,7739,2,No,12,"3,0",High,NaN,Low,NaN,5,"3,0",19,NaN,2,8,1977,No


In [33]:
print(limpieza.df.shape)
print(limpieza.df.shape)

(1614, 36)
(1614, 36)


In [34]:
print(limpieza.df[["DailyRate", "EmployeeNumber", "MonthlyIncome", "TotalWorkingYears", "Age", "HourlyRate", "StandardHours"]].info())
print(limpieza.df["HourlyRate"].unique())

print(limpieza.df[["DailyRate", "EmployeeNumber", "MonthlyIncome", "TotalWorkingYears", "Age", "HourlyRate", "StandardHours"]].info())
print(limpieza.df["HourlyRate"].unique())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1614 entries, 0 to 1613
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   DailyRate          1490 non-null   float64
 1   EmployeeNumber     1183 non-null   float64
 2   MonthlyIncome      771 non-null    float64
 3   TotalWorkingYears  1088 non-null   float64
 4   Age                1614 non-null   int64  
 5   HourlyRate         1530 non-null   float64
 6   StandardHours      419 non-null    float64
dtypes: float64(6), int64(1)
memory usage: 88.4 KB
None
[ 51.  65.  58.  82.  45.  99.  91.  64.  55.  68.  49.  61.  79.  31.
  69.  48.  80.  74.  98.  59.  33.  56.  66.  57.  53.  87.  81.  84.
  32.  41.  92.  47.  nan  43.  86.  30.  42.  88.  96.  67.  62.  72.
  78.  89.  52.  50.  90.  37.  94.  76.  60.  46.  83. 100.  40.  97.
  54.  75.  39.  85.  63.  44.  93.  36.  35.  73.  71.  70.  38.  77.
  95.  34.]
<class 'pandas.core.frame.DataFrame'>
R

In [35]:
print(limpieza.df["JobRole"].value_counts())

print(limpieza.df["JobRole"].value_counts())

JobRole
Sales Executive               369
Research Scientist            314
Laboratory Technician         278
Manufacturing Director        158
Healthcare Representative     149
Manager                       111
Sales Representative           90
Research Director              88
Human Resources                57
Name: count, dtype: int64
JobRole
Sales Executive               369
Research Scientist            314
Laboratory Technician         278
Manufacturing Director        158
Healthcare Representative     149
Manager                       111
Sales Representative           90
Research Director              88
Human Resources                57
Name: count, dtype: int64


In [29]:
limpieza.guardar_archivo()